In [125]:
import sys
modules_dir = 'D:/usr14/project/Binary/adiabaticbinary/ww'
sys.path.insert(0, modules_dir)

In [126]:
from modules import resnet1
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import wandb
import numpy as np
from readData import read_dataset

In [127]:
device=torch.device('cuda')
model=optim=lossfunc=train_loader=None
def fit():
    model.train()
    totalloss = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optim.zero_grad()
        output = model(data).to(device)
        loss = lossfunc(output, target)
        loss.backward()
        optim.step()
        with torch.no_grad():
            totalloss += loss.item()*data.size(0)
    return totalloss/len(train_loader.sampler)

def evaluate(val_loader: DataLoader):
    model.eval()
    loss=0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).to(device)
            loss+=lossfunc(outputs,labels).item()*inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct+=(predicted==labels).sum().item()

    return loss/len(val_loader.sampler) ,correct / total

In [128]:
model = resnet1.ResNet(True).to(device)
dicfull=torch.load('fullbest.pth')
dicbin=model.state_dict()
dicbin.update(dicfull)
model.load_state_dict(dicbin)
lossfunc=nn.CrossEntropyLoss().to(device)
train_loader,val_loader,test_loader=read_dataset(128)
# wb=None
wb=wandb.init(
    project="res cifar10",
    name="kkstep50",
    reinit=True,
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [129]:
target_acc=0.95
mepoch=9999999
mstep_kk=50
mstep_lr=10
lr=.1
kk_mul=1.5

In [130]:
_, vacc = evaluate(val_loader)
print(f"last fullbest:{vacc}")
while vacc>target_acc:
    model.set_kk(model.get_kk() * kk_mul)
    _, vacc = evaluate(val_loader)
    print(f"push kk to {model.get_kk()}, acc={vacc}")
kk_now, ka_now = model.toBin()
bloss, binbest = evaluate(val_loader)
print(f"last binbest:{binbest}, ka:{ka_now}, kk:{kk_now}")
model.quitBin()
if binbest >= target_acc:
    print("\033[91mtarget acc already reached")
    print("\033[0m")
step_kk = 0
step_lr = 0
vlbest = np.inf
vabest = 0
for epoch in range(mepoch):
    if step_lr / 10 == 1:
        step_lr = 0
        lr *= 0.5
        print(f"lr reduced to {lr}")
    optim = torch.optim.SGD(model.parameters(), lr, 0.9, weight_decay=5e-4)

    tloss = fit()
    vloss, vacc = evaluate(val_loader)


    if vloss < vlbest:
        step_kk-=(step_lr+1)
        step_lr = 0
        vlbest = vloss
    else:
        step_lr += 1

    model.toBin()
    bloss, bacc = evaluate(val_loader)
    if binbest < bacc:
        binbest = bacc
        print("\033[94mtest perf: ", end="")
        _, tacc = evaluate(test_loader)
        print(tacc)
        print("\033[0m")
        model.quitBin()
        torch.save(
            model.state_dict(),
            "binbest.pth",
        )
    else:
        model.quitBin()

    if binbest > target_acc:
        print("\033[91mtarget acc reached")
        print("\033[0m")
        break

    if vabest<vacc:
        vabest=vacc
        torch.save(model.state_dict(),'temp.pth')
    if step_kk >= mstep_kk:
        vacc=target_acc = vabest
        vabest=0
        step_kk = 0
        print(f"reduce acc to {target_acc}")

    print(
        "epoch",
        epoch + 1,
        "tloss",
        tloss,
        "vloss",
        vloss,
        "vacc",
        vacc,
        "bacc",
        bacc,
        "steplr",
        step_lr,
        "step_kk",
        step_kk,
        "lr",
        round(lr,5),
        "kk",
        round(model.get_kk(), 3),
        "tg",
        round(target_acc, 3),
    )
    if wb:
        wb.log(
            {
                "epoch":epoch+1,
                "train loss":tloss,
                "val acc":vacc,
                "bin acc":bacc,
                "lr": round(lr,5),
                "kk":round(model.get_kk(), 3),
                "tg":round(target_acc, 3),
            }
        )
    if vacc >= target_acc-1e-5:
        model.load_state_dict(torch.load("temp.pth"))
        step_kk = step_lr = vabest = 0
        vlbest = np.inf
        lr = 0.1
        while vacc > target_acc-1e-5:
            model.set_kk(model.get_kk() * kk_mul)
            _, vacc = evaluate(val_loader)
            print(f"push kk to {model.get_kk()}, acc={vacc}")
        torch.save(model.state_dict(),"afterpush.pth")
    else:
        step_kk += 1
if wb:
    wb.finish()

last fullbest:0.9904
push kk to 1.5, acc=0.1032
last binbest:0.1081, ka:None, kk:1.5
epoch 1 tloss 1.2156262439489365 vloss 0.9359786273956299 vacc 0.6744 bacc 0.1081 steplr 0 step_kk -1 lr 0.1 kk 1.5 tg 0.95
epoch 2 tloss 0.8838327435493469 vloss 0.9758108665466309 vacc 0.6792 bacc 0.1081 steplr 1 step_kk 0 lr 0.1 kk 1.5 tg 0.95
epoch 3 tloss 0.7632572838783264 vloss 1.0686024322509766 vacc 0.677 bacc 0.1081 steplr 2 step_kk 1 lr 0.1 kk 1.5 tg 0.95
epoch 4 tloss 0.7063957721710205 vloss 0.658629415512085 vacc 0.7763 bacc 0.1081 steplr 0 step_kk -1 lr 0.1 kk 1.5 tg 0.95
epoch 5 tloss 0.6659241627693177 vloss 0.6565278744697571 vacc 0.7816 bacc 0.0986 steplr 0 step_kk -1 lr 0.1 kk 1.5 tg 0.95
epoch 6 tloss 0.6345546607971191 vloss 0.5324524871826172 vacc 0.8214 bacc 0.1081 steplr 0 step_kk -1 lr 0.1 kk 1.5 tg 0.95
epoch 7 tloss 0.6108026289939881 vloss 0.8077975014686585 vacc 0.7365 bacc 0.0986 steplr 1 step_kk 0 lr 0.1 kk 1.5 tg 0.95
epoch 8 tloss 0.6022906518936157 vloss 0.47108110141

bin acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▄▄▁▇▇▄███████▇██▇██
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
kk,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▆▆▆███
lr,█▄▁█▄▁█▄▁█▄▂█▄▂██▂██▂█▄▂█▄▂█▄▃█▄▂▁█▂▁█▃▁
tg,█████████▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▇▇▇▆▆▆▅▅▅▃▃▃▃▁▁▁
train loss,█▅▁▇▅▁▇▅▁█▅▂▇▅▂▇▇▂▇▇▂▇▅▂▇▅▂▆▅▃▆▆▃▂▇▄▃▇▅▃
val acc,▃▄█▂▂█▄▅█▁▅▇▂▃▇▃▃▇▃▁▇▃▅█▃▅▇▃▅▆▄▅▇▇▁▆▇▂▄▆
bin acc,0.9206
epoch,1060
kk,194.62
lr,0.00313


In [143]:
model.load_state_dict(torch.load("binbest.pth"))
try:
    model.toBin()
except AssertionError:
    pass
evaluate(test_loader)

(0.25928978748321535, 0.9186)